# Peeking Duck: duckdb + lance for computer vision
`SELECT predict('resnet', image) FROM tbl`

Duckdb gives us the opportunity to simplify a huge part
of the ML workflow for computer vision. This notebook shows
how to use the Lance duckdb extension to run a pytorch
model on images in SQL:

```sql
SELECT filename, class, predict('resnet', image) as pred
FROM oxford_pet
WHERE split='train' AND class='samoyed'
USING SAMPLE 1000;
```

This is made possible using DuckDB in conjunction with Lance, <br>
a new columnar data format for computer vision (CV). <br>
Lance is like Parquet but built with CV in mind, <br>
with fast point-access, partial reads and optimisations for nested annotation columns. 

tl;dr - Lance is a more performant and CV-specific parquet.

Lance can be accessed by tools like Pandas and DuckDB via Apache Arrow. Let's see it in action.

For reference:
1. The Lance file format lives [here](https://github.com/eto-ai/lance)
2. The Lance duckdb extension is under the [/integrations/duckdb](https://github.com/eto-ai/lance/tree/main/integration/duckdb) subdirectory in Lance

## Model setup
It only takes a few lines of code to prepare a model for inference

### Creating the model

Convert a pre-trained resnet to torchscript and save it

In [13]:
import torch
from torchvision.models import resnet50, ResNet50_Weights

resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
m = torch.jit.script(resnet)
torch.jit.save(m, '/tmp/model.pth')

### Setup the duckdb extension

For now, the extension should be [built from source](https://github.com/eto-ai/lance/tree/main/integration/duckdb#development).

Once that's done, we can install and load the extensio. In this example we copied the artifact `lance.duckdb_extension` into the same directory as this notebook. Instead, you can also just supply it with a relative path in the `install_extension` call below.

In [9]:
import torch
import duckdb
con = duckdb.connect(config={"allow_unsigned_extensions": True})
con.install_extension("lance.duckdb_extension", force_install=True)
con.load_extension("lance")

### Load the torchscript model
For each model that we saved as torchscript, <br>
use the `create_pytorch_model` function to register the model <br> 
from where we saved it. We can list all registered functions <br>
using the `ml_models` table function.

In [2]:
con.query("CALL create_pytorch_model('resnet', '/tmp/model.pth');")
con.query("SELECT * FROM ml_models();").to_df()

,name,uri,type
0,resnet,/tmp/model.pth,torchscript


## Load the dataset

Resnet was trained on ImageNet, but it'd be fun to run <br>
it on a different dataset. Here we use the [Oxford Pet
dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/).<br>


The raw dataset is organized into
```
/images
/annotations
```
and /annotations has
1. data indices: list.txt, test.txt, trainval.txt
2. /xmls annotations in pascal voc format
3. /trimap of trimap (png's)

To make it queryable, we've converted it into a Lance dataset on a public s3 bucket

In [3]:
from lance import LanceFileFormat
import pyarrow.dataset as ds
uri ="s3://eto-public/datasets/oxford_pet/oxford_pet.lance"
oxford_pet = ds.dataset(uri, format=LanceFileFormat())
print(oxford_pet.schema.names)

['_pk', 'filename', 'class', 'species', 'breed', 'split', 'folder', 'source', 'size', 'segmented', 'object', 'external_image', 'image']


If you're interested in more details on `LanceFileFormat`, <br>
head over to [Lance github page](https://github.com/eto-ai/lance) or stay tuned for another post!

## Query the data
You can use duckdb to query Lance data via Apache Arrow

In [4]:
from lance.types import Image

df = con.query("""
SELECT external_image as image_uri
FROM oxford_pet
WHERE class='samoyed'
LIMIT 10;
""").to_df()

Image.create(df.image_uri[0])

Image(s3://eto-public/datasets/oxford_pet/images/samoyed_100.jpg)

## Let's make a prediction
Now it's time to use the registered resnet model to do inference

In [8]:
predictions = con.query("""
SELECT class, list_argmax(predict('resnet', image)) as pred
FROM oxford_pet
WHERE split='train' AND class='samoyed'
LIMIT 10;
""").to_df()
predictions

,class,pred
0,samoyed,258
1,samoyed,258
2,samoyed,258
3,samoyed,258
4,samoyed,258
5,samoyed,258
6,samoyed,258
7,samoyed,258
8,samoyed,258
9,samoyed,258


`predict` allows you to invoke a registered model by name <br>
`image` is a binary column of the image bytes <br>
`list_argmax` finds the position with the highest output probability

### What does 258 mean?
How do we know if the predictions are reasonable?

In [11]:
import pandas as pd

labels_uri = ("https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels"
              "/master/imagenet-simple-labels.json")

labels = (pd.read_json(labels_uri)
          .reset_index()
          .rename(columns={0: "label", "index": "label_id"}))

In [12]:
con.query(
    """
    SELECT 
      predictions.class as gt_label,
      labels.label as pred_label
    FROM predictions 
      INNER JOIN labels on predictions.pred=labels.label_id;
    """
).to_df()

,gt_label,pred_label
0,samoyed,Samoyed
1,samoyed,Samoyed
2,samoyed,Samoyed
3,samoyed,Samoyed
4,samoyed,Samoyed
5,samoyed,Samoyed
6,samoyed,Samoyed
7,samoyed,Samoyed
8,samoyed,Samoyed
9,samoyed,Samoyed


## Conclusion

As you can see from this notebook, Lance makes it easy to do analytics and model inference in SQL by using the Lance extension for duckdb. With Lance (via Arrow), we can manage images, metadata, and annotations all in one place, and we can query it efficiently even when the data lives in cheap remote storage.

You can find Lance here: https://github.com/eto-ai/lance If you like us, we'd love a star on our project, and we'd appreciate your feedback even more!